In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

from importlib import reload
#syntax: fugi = reload(fugi)

import functions_giammi as fugi

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import re

#for the cartesian product
import itertools

#to smooth the MFPADs
import scipy as sp
import scipy.ndimage

#to create the mesh
import triangulation as tr
from scipy.spatial import Delaunay
from pyntcloud import PyntCloud, structures

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

##550eV - 11.5eV CH11
#update nov Kilian binning (36,18)
fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CR_Knov.root");en="550eV";CH="CH11";
fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CL_Knov.root")
fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CR_Knov.root")
fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CL_Knov.root")

# #546eV - 8.2eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CR_10800-2350ns_multiCH11.root");en="546eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CL_10800-2350ns_multiCH11.root")

# 544eV - 6.1eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CR_9600-3700ns_multiCH11.root");en="544eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CL_9600-3700ns_multiCH11.root")

# #542eV - 4eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CR_10800-2350ns_COMBI_multiCH11.root");en="542eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CL_10800-2350ns_COMBI_multiCH11.root")

# #541eV 3eV CH11
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CR_10800-2350ns_multiCH11.root");en="541eV";CH="CH11";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CL_10800-2350ns_multiCH11.root")

#Select between CH9 and CH11
path = "angular_distr_el/"+CH+"/"

#move first the ID you need
IDs = ["ID_ALL_mol_e0_valid/EN_gate/","ID1_mol_e0_valid/EN_gate/","ID2_mol_e0_valid/EN_gate/","ID3_mol_e0_valid/EN_gate/"]
dirs = ["MFPADs_multinew","MFPADs_multinew_phi"]

loc=path+IDs[0]+dirs[0]

#to alow the helicity comparison without the S enantiomer
if en != "550eV":
    fileSCR = None
    fileSCL = None

#new uproot4
# https://uproot.readthedocs.io/en/latest/basic.html
d=dict(fileRCR[loc].classnames()) # shows the type of each item
# k=fileRCR[loc].keys() #same as .classnames()
check = fileRCR[loc+"/MFPAD3D_engate_costheta_-1.00_phi_-180"]
d1=type(check)
check.all_members; #all members of the single histogram

In [ ]:
#Loading of the MFPADS and cos len = 27
MFPAD_RCR,ctheta_RCR,cosphi_photon,MFPAD_axis,ctheta_axis,ctheta_axis_red=fugi.import_MFPAD(fileRCR, loc, full=True)
MFPAD_RCL,ctheta_RCL=fugi.import_MFPAD(fileRCL, loc)

if fileSCL and fileSCR is not None:    
    MFPAD_SCR,ctheta_SCR=fugi.import_MFPAD(fileSCR, loc)
    MFPAD_SCL,ctheta_SCL=fugi.import_MFPAD(fileSCL, loc)

In [ ]:
#COORDIANTES PHOTON ROTATION
#adjusting cosphi with a shifting to have it equal to theory
#before sorting it is natuarally cos sorted from experiment
cosphi_adj=[]
ctheta_axis_red=ctheta_axis_red[0];

phiM=(MFPAD_axis[0][0][1:] + MFPAD_axis[0][0][:-1])/2
cosM=(MFPAD_axis[0][1][1:] + MFPAD_axis[0][1][:-1])/2

#cartesian product of unique values, (288,2) each
phicos_MFPAD = np.array(list(itertools.product(phiM, cosM)))
cosphi_MFPAD = np.array(list(itertools.product(cosM, phiM)))

#(288,1) for each
ctheta_full = np.array([col[0] for col in cosphi_MFPAD])
theta_rad = np.arccos(ctheta_full)

phi_full = np.array([col[1] for col in cosphi_MFPAD])
phi_rad = phi_full * np.pi/180.

#effective dimensions to avoid retyping
c=cosM.shape[0]
p=phiM.shape[0]

print("MFPAD_xy shape = ", MFPAD_axis[0][0].shape)
print("phiM shape = ", phiM.shape)
print("cosM shape = ", cosM.shape)

print("cosphi diagnostic: type=",type(cosphi_photon), " shape= ", cosphi_photon.shape)

cos_adj=fugi.shift_func(cosphi_photon[:,0]).tolist();
phi_adj=fugi.shift_func(cosphi_photon[:,1]).tolist();

for cos,phi in zip(cos_adj,phi_adj):
    cosphi_adj.append((cos,phi))
cosphi_adj=np.around(np.array(cosphi_adj),3)

print("cosphi_adj: type=",type(cosphi_adj), " shape= ", cosphi_adj.shape)

print(cosphi_adj[0][0])
#look to the values in b1 map and found the MFPAD

dfind=pd.DataFrame(cosphi_adj, columns=["ctheta","phi"])

#rearranging the photon vector
cosphi_adj_cos=(dfind.sort_values(by=["ctheta", "phi"])).values
cosphi_adj_phi=(dfind.sort_values(by=["phi","ctheta"])).values

In [ ]:
#import 3d graph kilian
MFPAD_RCLtest=fugi.import_MFPAD3D(fileRCL, loc)
MFPAD_RCRtest=fugi.import_MFPAD3D(fileRCR, loc)

#sum
MFPAD_RCLtest_sum=np.sum(MFPAD_RCLtest,axis=0)
MFPAD_RCRtest_sum=np.sum(MFPAD_RCRtest,axis=0)

#normalise each and sum
# MFPAD_RCLtest_sum=np.sum(fugi.normalise_matrix(MFPAD_RCLtest),axis=0)
# MFPAD_RCRtest_sum=np.sum(fugi.normalise_matrix(MFPAD_RCRtest),axis=0)
MFPAD_Rtest_sum = np.add(MFPAD_RCLtest_sum,MFPAD_RCRtest_sum)

if fileSCL and fileSCR is not None:
    MFPAD_SCRtest=fugi.import_MFPAD3D(fileSCR, loc)
    MFPAD_SCLtest=fugi.import_MFPAD3D(fileSCL, loc)
    MFPAD_SCRtest_sum=np.sum(MFPAD_SCRtest,axis=0)
    MFPAD_SCLtest_sum=np.sum(MFPAD_SCLtest,axis=0)

    # MFPAD_SCLtest_sum=np.sum(fugi.normalise_matrix(MFPAD_SCLtest),axis=0)
    # MFPAD_SCRtest_sum=np.sum(fugi.normalise_matrix(MFPAD_SCRtest),axis=0)

    #flip along cos(theta)
    MFPAD_SCLtest_flip_sum=np.flip(MFPAD_SCLtest_sum,axis=0)
    MFPAD_SCRtest_flip_sum=np.flip(MFPAD_SCRtest_sum,axis=0)
    MFPAD_Stest_sum = np.add(MFPAD_SCLtest_flip_sum,MFPAD_SCRtest_flip_sum)

    MFPAD_tottest=np.add(MFPAD_Stest_sum,MFPAD_Rtest_sum)

    print(MFPAD_tottest.shape)
else:
    MFPAD_tottest=MFPAD_Rtest_sum

In [ ]:
#normalising entries, important just for PECD
MFPAD_RCR_norm = fugi.normalise_matrix(MFPAD_RCR,normtype=2) # z values normalise to number of event
MFPAD_RCL_norm = fugi.normalise_matrix(MFPAD_RCL,normtype=2) # z values normalise to number of event

if fileSCL and fileSCR is not None:
    MFPAD_SCR_norm = fugi.normalise_matrix(MFPAD_SCR,normtype=2) # z values normalise to number of event
    MFPAD_SCL_norm = fugi.normalise_matrix(MFPAD_SCL,normtype=2) # z values normalise to number of event


In [ ]:
#flipping; axis=1 flips along cos(theta), axis=0 along phi
MFPAD_RCL_flip=[]
MFPAD_RCR_flip=[]

#flipping sorted S enantiomer for R enantiomer: JUST ALONG COS(theta) -> axis=1
for el in MFPAD_RCL:
    MFPAD_RCL_flip.append(np.flip(el, axis=1))
MFPAD_RCL_flip=np.array(MFPAD_RCL_flip)

#flipping sorted S enantiomer for R enantiomer: JUST ALONG COS(theta) -> axis=1
for el in MFPAD_RCR:
    MFPAD_RCR_flip.append(np.flip(el, axis=1))
MFPAD_RCR_flip=np.array(MFPAD_RCR_flip)

if fileSCL and fileSCR is not None:
    MFPAD_SCL_flip=[]
    MFPAD_SCR_flip=[]

    for el in MFPAD_SCL:
        MFPAD_SCL_flip.append(np.flip(el, axis=1))
    MFPAD_SCL_flip=np.array(MFPAD_SCL_flip)

    for el in MFPAD_SCR:
        MFPAD_SCR_flip.append(np.flip(el, axis=1))
    MFPAD_SCR_flip=np.array(MFPAD_SCR_flip)    

In [ ]:
#sum within an experimemt axis=0 is among the 72 elements
#single enantiomer and helicity
#sum all MFPAD: NO NORMALIZATION
MFPAD_RCR_sum=np.sum(MFPAD_RCR, axis=0)
MFPAD_RCL_sum=np.sum(MFPAD_RCL, axis=0)
MFPAD_R_sum=np.add(MFPAD_RCR_sum,MFPAD_RCL_sum)

if fileSCL and fileSCR is not None:
    #single enantiomer and helicity
    MFPAD_SCR_sum=np.sum(MFPAD_SCR, axis=0)
    MFPAD_SCL_sum=np.sum(MFPAD_SCL, axis=0)
    MFPAD_SCR_flip_sum=np.sum(MFPAD_SCR_flip, axis=0)
    MFPAD_SCL_flip_sum=np.sum(MFPAD_SCL_flip, axis=0)

    MFPAD_S_sum=np.add(MFPAD_SCR_flip_sum,MFPAD_SCL_flip_sum)

    #sum of two enantiomers: there is no influence of the ellicity
    MFPAD_tot_sumt=MFPAD_RCR_sum+MFPAD_RCL_sum+MFPAD_SCR_flip_sum+MFPAD_SCL_flip_sum #bug?
    MFPAD_tot_sum=np.add(MFPAD_R_sum,MFPAD_S_sum)
else:
    MFPAD_tot_sum=MFPAD_R_sum

In [ ]:
#sorting
#let´s work just with NOT NORMALISED becuase of MFPAD
MFPAD_RCR_cos=fugi.sorting_array(MFPAD_RCR, theory=False, a=1)
MFPAD_RCL_cos=fugi.sorting_array(MFPAD_RCL, theory=False, a=1)
MFPAD_RCR_phi=fugi.sorting_array(MFPAD_RCR, theory=False, a=2)
MFPAD_RCL_phi=fugi.sorting_array(MFPAD_RCL, theory=False, a=2)

MFPAD_RCL_flip_cos=fugi.sorting_array(MFPAD_RCL_flip, theory=False, a=1)
MFPAD_RCL_flip_phi=fugi.sorting_array(MFPAD_RCL_flip, theory=False, a=2)
MFPAD_RCR_flip_cos=fugi.sorting_array(MFPAD_RCR_flip, theory=False, a=1)
MFPAD_RCR_flip_phi=fugi.sorting_array(MFPAD_RCR_flip, theory=False, a=2)


if fileSCL and fileSCR is not None:
    MFPAD_SCR_cos=fugi.sorting_array(MFPAD_SCR, theory=False, a=1)
    MFPAD_SCL_cos=fugi.sorting_array(MFPAD_SCL, theory=False, a=1)
    MFPAD_SCR_phi=fugi.sorting_array(MFPAD_SCR, theory=False, a=2)
    MFPAD_SCL_phi=fugi.sorting_array(MFPAD_SCL, theory=False, a=2)

    MFPAD_SCL_flip_cos=fugi.sorting_array(MFPAD_SCL_flip, theory=False, a=1)
    MFPAD_SCL_flip_phi=fugi.sorting_array(MFPAD_SCL_flip, theory=False, a=2)
    MFPAD_SCR_flip_cos=fugi.sorting_array(MFPAD_SCR_flip, theory=False, a=1)
    MFPAD_SCR_flip_phi=fugi.sorting_array(MFPAD_SCR_flip, theory=False, a=2)

In [ ]:
#smart flipping for the single 72 MFPAD image. Has to be in cos, do the phi just as a check
MFPAD_RCLCR=[]

#sum helicities: -cos(theta), phi+180 in photon coordiantes
for j in range(12):
    for i in range(6):
        if j>5:
            MFPAD_RCLCR.append(np.add(MFPAD_RCL_phi.reshape(6,12,-1)[i,j],MFPAD_RCR_phi.reshape(6,12,-1)[5-i,j-6]))
            i+=1
        else:
            MFPAD_RCLCR.append(np.add(MFPAD_RCL_phi.reshape(6,12,-1)[i,j],MFPAD_RCR_phi.reshape(6,12,-1)[5-i,6+j]))
            i+=1
MFPAD_RCLCR=np.array(MFPAD_RCLCR).reshape(72,MFPAD_RCL_phi.shape[1],MFPAD_RCL_phi.shape[2])
MFPAD_RCLCR_sum=np.sum(MFPAD_RCLCR, axis=0)

#sorting
MFPAD_RCLCR_cos=fugi.sorting_array(MFPAD_RCLCR, theory=False, a=1)
MFPAD_RCLCR_phi=fugi.sorting_array(MFPAD_RCLCR, theory=False, a=2) #equal to the original!

if fileSCL and fileSCR is not None:
    MFPAD_SCLCR=[]
    MFPAD_totordered=[]
    #sum helicities: -cos(theta), phi+180 in photn coordiantes
    for j in range(12):
        for i in range(6):
            if j>5:
                MFPAD_SCLCR.append(np.add(MFPAD_SCL_phi.reshape(6,12,-1)[i,j],MFPAD_SCR_phi.reshape(6,12,-1)[5-i,j-6]))
                i+=1
            else:
                MFPAD_SCLCR.append(np.add(MFPAD_SCL_phi.reshape(6,12,-1)[i,j],MFPAD_SCR_phi.reshape(6,12,-1)[5-i,6+j]))
                i+=1
    MFPAD_SCLCR=np.array(MFPAD_SCLCR).reshape(72,MFPAD_SCL_phi.shape[1],MFPAD_SCL_phi.shape[2])
    MFPAD_SCLCR_sum=np.sum(MFPAD_SCLCR, axis=0)

    #sorting
    MFPAD_SCLCR_cos=fugi.sorting_array(MFPAD_SCLCR, theory=False, a= 1)
    MFPAD_SCLCR_phi=fugi.sorting_array(MFPAD_SCLCR, theory=False, a= 2) #equal to the original!
    
    #sum enantiomer: -cos(theta) fliped axis=1 cos!, -cos(theta) photon coordinates
    nphi=MFPAD_SCL.shape[1];ncos=MFPAD_SCL.shape[2]
    for j in range(12):
        for i in range(6):
            MFPAD_totordered.append(np.add(MFPAD_SCLCR_phi.reshape(6,12,nphi,ncos)[i,j],np.flip(MFPAD_RCLCR_phi.reshape(6,12,nphi,ncos)[5-i,j], axis=1)))
            i+=1

    MFPAD_totordered=np.array(MFPAD_totordered).reshape(72,MFPAD_SCL.shape[1],MFPAD_SCL.shape[2])
    
    #sorting
    MFPAD_totordered_cos=fugi.sorting_array(MFPAD_totordered, theory=False, a=1)
    MFPAD_totordered_phi=fugi.sorting_array(MFPAD_totordered, theory=False, a=2) #equal to the original!

    #PECD check it is equal to contrast
    mPECD = np.divide(np.subtract(MFPAD_RCLCR_sum,MFPAD_SCLCR_sum),np.add(MFPAD_RCLCR_sum,MFPAD_SCLCR_sum))

else:
    MFPAD_totordered=MFPAD_RCLCR
    MFPAD_totordered_cos=MFPAD_RCLCR_cos
    MFPAD_totordered_phi=MFPAD_RCLCR_phi
    mPECD=np.divide(np.subtract(MFPAD_RCL_sum,MFPAD_RCR_sum),np.add(MFPAD_RCL_sum,MFPAD_RCR_sum))

In [ ]:
#COORDIANTES PHOTON ROTATION
#adjusting cosphi with a shifting to have it equal to theory
ctheta_axis_red=ctheta_axis_red[0]; #Ffrom ccos(theta)

phiM=(MFPAD_axis[0][0][1:] + MFPAD_axis[0][0][:-1])/2 #phi = (36,), MFPAD_axis = (37,)
cosM=(MFPAD_axis[0][1][1:] + MFPAD_axis[0][1][:-1])/2 #COS = (36,), MFPAD_axis = (37,)

print("MFPAD_xy shape = ", MFPAD_axis[0][0].shape)
print("phiM shape = ", phiM.shape)
print("cosM shape = ", cosM.shape)

In [ ]:
#look to the values in b1 map and found the MFPAD. The phi - cos(theta) grid of light direction id indipendendt from the enantiomer
#the number of the element depends on the sorting of the b1 map at which the maximum has been calculated. Usually is phi sorted.
dfind=pd.DataFrame(cosphi_adj_phi, columns=["ctheta","phi"])
dfind.loc[dfind["phi"] == -180]

In [ ]:
#insert hier the corrispondent index number coming from the comparison with the b1 map values
#NOTE: sort the MFPAD before choosing
checkthis=23

In [ ]:
#select here the MFPAD to check

#from spherical to cartesian coordinates
counts=MFPAD_totordered_phi[checkthis].reshape(-1)
x = counts * np.sin(theta_rad) * np.cos(phi_rad)
X = x.reshape(p,c)
y = counts * np.sin(theta_rad) * np.sin(phi_rad)
Y = y.reshape(p,c)
z = counts * np.cos(theta_rad)
Z = z.reshape(p,c)

d = np.sqrt(x**2+y**2+z**2)
d_matrix = np.sqrt(X**2+Y**2+Z**2)

In [ ]:
#from spherical to cartesian coordinates
#SMOOTHED by sigmax (ctheta) and sigmay (phi)
# sigma phi (y) can be double the value of x

counts=fugi.smoothgauss(MFPAD_totordered_phi[checkthis],0.3,0.6).reshape(-1)
xs = counts * np.sin(theta_rad) * np.cos(phi_rad)
Xs = xs.reshape(p,c)
ys = counts * np.sin(theta_rad) * np.sin(phi_rad)
Ys = ys.reshape(p,c)
zs = counts * np.cos(theta_rad)
Zs = zs.reshape(p,c)

ds = np.sqrt(xs**2+ys**2+zs**2)
d_matrixs = np.sqrt(Xs**2+Ys**2+Zs**2)

In [ ]:
fig1 = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers',
                    marker=dict( size=5,color=d, colorscale='Viridis', opacity=1., showscale=True)
                    # color set colours to an array/list of desired values  
                    # show scale to show the legend according to the color
                    )])

fig1.update_layout(title="TFMeOx MFPADs EXP", width=500, height=500, margin=dict(l=65, r=50, b=65, t=90)) #margin=dict(l=0, r=0, b=0, t=0))
fig1.show()

fig1s = go.Figure(data=[go.Scatter3d(x=xs, y=ys, z=zs, mode='markers',
                    marker=dict( size=20,color=ds, colorscale='Viridis', opacity=1., showscale=True)
                    # color set colours to an array/list of desired values  
                    # show scale to show the legend according to the color
                    )])

fig1s.update_layout(title="TFMeOx MFPADs EXP smoothed", width=500, height=500, margin=dict(l=0, r=0, b=0, t=0)) #margin=dict(l=0, r=0, b=0, t=0))
fig1s.show()

In [ ]:
delaunay_tri, point_trace, my_mesh3d, delaun_tri3d = fugi.makeamesh(xs,ys,zs,ds)

fig2 = go.Figure(data=[delaunay_tri, point_trace])
fig2.update_layout(width=500, height=500);
# fig2.show()

my_layout = dict(width=800, height=600,
                 scene_camera_eye=dict(x=1.5, y=1.5, z=1), 
                 scene_aspectratio=dict(x=1.5, y=1.5, z=1))

fig2s = go.Figure(data=[my_mesh3d, delaun_tri3d])
# fig2s.show()

In [ ]:
xn,yn,zn,In,Jn,Kn = fugi.getamesh(x,y,z,d)

fig3 = go.Figure(go.Mesh3d(x=x, y=y, z=z,
                           alphahull=5,  
                        #    i=In, j=Jn, k=Kn, 
                           intensity=d,
                           colorscale="Viridis",
                           colorbar_len=0.75,
                           flatshading=True,
                           lighting=dict(ambient=0.5,
                                         diffuse=1,
                                         fresnel=4,        
                                         specular=0.5,
                                         roughness=0.05,
                                         facenormalsepsilon=0,
                                         vertexnormalsepsilon=0),
                          lightposition=dict(x=100,
                                             y=100,
                                            z=1000)))
fig3.update_layout(width=800, height=800)
# fig1.show()


In [ ]:
fig4 = go.Figure(data=[go.Surface(z=Z, surfacecolor=d_matrix)])
fig4.update_layout(title='TFMeOx MFPADs EXP surf', autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig4.show()

fig4s = go.Figure(data=[go.Surface(z=Zs, surfacecolor=d_matrixs)])
fig4s.update_layout(title='TFMeOx MFPADs EXP surf SMOOTHED', autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig4s.show()


In [ ]:
def lod_mesh_export(mesh, lods, extension, path):
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh(path+"lod_"+str(i)+extension, mesh_lod)
        mesh_lods[i]=mesh_lod
    print("generation of "+str(i)+" LoD successful")
    return mesh_lods

In [ ]:
import open3d as o3d
import trimesh

sys.path.append('..')
output_path=(r"../PYTHON_graphs/OUTPUTS//")

alpha_mesh=[]
poisson_mesh=[]
densities=[]

alpha_mesh = o3d.geometry.TriangleMesh()
pcd = o3d.geometry.PointCloud()
pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3)))  # invalidate existing normals

camera_loc=np.array([0.,0.,-1.])


#load the point cloud
# SMOOTHED
point_cloud=np.array([xs,ys,zs]).T
#Standard
# point_cloud=np.array([x,y,z]).T
cloud = PyntCloud.from_instance("open3d", pcd)
pcd.points = o3d.utility.Vector3dVector(point_cloud)
#resise the scale of the sample
vox_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, 1.)

#presetn in all approaches of plc
kdtree = cloud.add_structure("kdtree")
testc = cloud.get_neighbors(k=5)
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)

# mesh.compute_vertex_normals()


#compute the normals
pcd.estimate_normals(); #mandatory
# pcd.normals = o3d.utility.Vector3dVector(point_cloud) #perfectly radial and long, independent by the tangent plane
# They see good for open surfaces, but not for my case
# pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05,max_nn=20));
# pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50));

#orient the normals
#Number of nearest neighbours: 5 is the minimum to have a closed surface with scale >= 2
#high bins
pcd.orient_normals_consistent_tangent_plane(7)

#normal bins
# pcd.orient_normals_consistent_tangent_plane(5)
# pcd.orient_normals_towards_camera_location(camera_loc)
# pcd.orient_normals_to_align_with_direction([0.,1.,0.])


#alpha-shape algorithm
# alpha=np.log10(0.5)
# alpha=0.1
# tetralpha_mesh, pt_map = o3d.geometry.TetraMesh.create_from_point_cloud(pcd)
# alpha_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha, tetralpha_mesh, pt_map) #it resturns zero for some reason


#Poisson algorithm
#high bins
poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9, width=0, scale=2., linear_fit=False)
# poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9, width=0, scale=2., linear_fit=False)

#normal bins
# poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9, width=0, scale=1.5, linear_fit=False)
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)
# p_mesh_crop.compute_triangle_normals() # usually computed before rendering (??)


#Hull ball algoritm
radii = [0.08, 0.1, 0.2, 0.4]
radius = 1.5 * avg_dist
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))
# bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector(radii))


'''
CLEANING THE TRIANGLES
arget_number_of_triangles (int) – The number of triangles that the simplified mesh should have. It is not guaranteed that this number will be reached.
maximum_error (float, optional, default=inf) – The maximum error where a vertex is allowed to be merged
boundary_weight (float, optional, default=1.0) – A weight applied to edge vertices used to preserve boundaries
'''
dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)
dec_mesh.remove_degenerate_triangles()
# dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
# dec_mesh.remove_non_manifold_edges()

#none of them is working with poisson
# p_mesh_crop =poisson_mesh.simplify_quadric_decimation(6000)
# p_mesh_crop.remove_unreferenced_vertices
# p_mesh_crop.remove_degenerate_triangles()
# p_mesh_crop.remove_duplicated_triangles()
# p_mesh_crop.remove_duplicated_vertices()
# p_mesh_crop.remove_non_manifold_edges()

#create the triangular mesh with the vertices and faces from open3d to be visualise with another software
tri_mesh = trimesh.Trimesh(np.asarray(dec_mesh.vertices), np.asarray(dec_mesh.triangles),vertex_normals=np.asarray(dec_mesh.vertex_normals))
tri_mesh_pois = trimesh.Trimesh(np.asarray(poisson_mesh.vertices), np.asarray(poisson_mesh.triangles),vertex_normals=np.asarray(poisson_mesh.vertex_normals))
# tri_mesh_pois_crop = trimesh.Trimesh(np.asarray(p_mesh_crop.vertices), np.asarray(p_mesh_crop.triangles),vertex_normals=np.asarray(p_mesh_crop.vertex_normals))
# trimesh.convex.is_convex(dec_mesh)

In [ ]:
type(tri_mesh_pois)
tri_mesh_pois.vertices

In [ ]:
# np.asarray(poisson_mesh.vertices).T
xgo, ygo, zgo =np.asarray(poisson_mesh.vertices).T

dgo = np.sqrt(xgo**2+ygo**2+zgo**2)

# asarray(p_mesh_crop.triangles)
# txgo, tygo, tzgo =np.asarray(p_mesh_crop.vertices).T[0]
# tdgo = np.sqrt(txgo**2+tygo**2+tzgo**2)

# np.asarray(poisson_mesh.vertex_normals)
# poisson_mesh.triangle_normals
tigo, tjgo, tkgo =np.asarray(poisson_mesh.triangles).T
ti, tj, tk = tri_mesh_pois.triangles.T

In [ ]:
#designing the surface colour
#densities are the real density of features
densities = np.asarray(densities)
density_colors = plt.get_cmap('viridis')((dgo - dgo.min()) / (dgo.max() - dgo.min()))
density_colors = density_colors[:, :3]

#works for the plotting in o3d
poisson_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)


o3d.io.write_triangle_mesh(output_path+"bpa_mesh.ply", dec_mesh);
o3d.io.write_triangle_mesh(output_path+"p_mesh_c.ply", poisson_mesh);
# o3d.io.write_triangle_mesh(output_path+"p_mesh_c.ply", p_mesh_crop);

# my_lods = lod_mesh_export(p_mesh_crop, [100000,50000,10000,1000,100], ".ply", output_path)
my_lods = lod_mesh_export(poisson_mesh, [100000,50000,10000,1000,100], ".ply", output_path)

# o3d.visualization.draw_geometries([pcd, p_mesh_crop], mesh_show_back_face=True)
# o3d.visualization.draw_geometries([pcd, poisson_mesh],mesh_show_back_face=True)
# o3d.visualization.draw_geometries([pcd, poisson_mesh[100000]],point_show_normal=True)

# tri_mesh_pois.show()

In [ ]:
fig5 = go.Figure(go.Mesh3d(x=xgo, y=ygo, z=zgo, 
                           # alphahull=3.5, 
                           i=tigo, j=tjgo, k=tkgo, 
                           # i=ti, j=tj, k=tk, 
                           intensity=dgo,
                           colorscale="Viridis",
                           colorbar_len=0.75,
                           flatshading=True,
                           lighting=dict(ambient=0.5,
                                         diffuse=1,
                                         fresnel=4,        
                                         specular=0.5,
                                         roughness=0.05,
                                         facenormalsepsilon=0,
                                         vertexnormalsepsilon=0),
                          lightposition=dict(x=100,
                                             y=100,
                                            z=1000)))
fig5.update_layout(width=800, height=800)
fig5.show()


In [ ]:
# Go scatters do not provide a legend
fig6 = go.Figure(data=[go.Scatter3d(x=xgo, y=ygo, z=zgo,
                    mode='markers',
                    marker=dict(
                        size=5,
                        color=dgo,            # set color to an array/list of desired values
                        colorscale='Viridis',   # choose a colorscale
                        opacity=1.,
                        showscale=True          # to show the legend according to the color
                   )
                )])

fig6.update_layout(title="TFMeOx MFPADs poisson scattered",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
                #   margin=dict(l=0, r=0, b=0, t=0))
fig6.show()

# fig6 = go.Figure(data=[go.Scatter3d(x=xs, y=ys, z=zs,
fig6s = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                    mode='markers',
                    marker=dict(
                        size=5,
                        # color=ds,
                        color=d,
                        colorscale='Viridis',   # choose a colorscale
                        opacity=1.,
                        showscale=True          # to show the legend according to the color
                   )
                )])

fig6s.update_layout(title="TFMeOx MFPADs smoothed scattered",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
                #   margin=dict(l=0, r=0, b=0, t=0))

fig6s.show()

In [ ]:
# tigo=np.asarray(poisson_mesh.vertex_normals).T[0]
np.asarray(poisson_mesh.vertex_normals)

In [ ]:
tri_mesh_pois.vertex_normals

In [ ]:
#SHORTCUTS from keyboard: n = show normals, q = quit, w = mesh
# o3d.visualization.draw_geometries([pcd, poisson_mesh],mesh_show_back_face=True)

In [ ]:
def rot3d(alpha,beta,gamma):  #planar rotation of alpha radians
    return np.array([[np.cos(alpha)*np.cos(beta), np.cos(alpha)*np.sin(beta)*np.sin(gamma)-np.sin(alpha)*np.cos(gamma), np.cos(alpha)*np.sin(beta)*np.cos(gamma)+np.sin(alpha)*np.sin(gamma)], 
                      [np.sin(alpha)*np.cos(beta), np.sin(alpha)*np.sin(beta)*np.sin(gamma)+np.cos(alpha)*np.cos(gamma), np.sin(alpha)*np.sin(beta)*np.cos(gamma)-np.cos(alpha)*np.sin(gamma)],
                      [-np.sin(beta),              np.cos(beta)*np.sin(gamma),                                           np.cos(beta)*np.cos(gamma)]])

frames=[]

fig8 = go.Figure(go.Mesh3d(x=xgo, y=ygo, z=zgo,i=tigo, j=tjgo, k=tkgo, intensity=dgo,
                           colorscale="Viridis",
                           colorbar_len=0.75,
                           flatshading=True,
                           lighting=dict(ambient=0.5,diffuse=1,fresnel=4,specular=0.5,roughness=0.05,facenormalsepsilon=0,vertexnormalsepsilon=0),lightposition=dict(x=100,y=100,z=1000)))

T = np.arange(0,  2*np.pi, 0.125)
xyz = np.stack((xgo, ygo, zgo))
for alpha in T:
    xr, yr, zr  =  np.einsum('ik, kj -> ij', rot3d(alpha,0.25*np.pi,0.), xyz)#  batch rotation of all points (x,y,z) on the Mesh
    frames.append(go.Frame(data=[go.Mesh3d(x=xr, y=yr, z=zr)])) #z is the same in a 3d rotation about zaxis
fig8.update(frames=frames);    


fig8.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)

fig8.update_layout(updatemenus=[dict(type='buttons',
                  showactive=False,
                  y=0.7,
                  x=-0.15,
                  xanchor='left',
                  yanchor='bottom',
                  buttons=[dict(label='Play',
                                 method='animate',
                                 args=[None, dict(frame=dict(duration=5, redraw=True), 
                                                             transition=dict(duration=4),
                                                             fromcurrent=True,
                                                            #  mode='immediate'
                                                            )]
                                            )
                                      ]
                              )
                        ])

# fig8.show()
# plotly.offline.plot(fig8, filename=r'../PYTHON_graphs/OUTPUTS/3D_exp_th_frames_n_highbins2.html')

In [ ]:
# recall variables from 3D_MFPDAs_theory
%store -r X
%store -r Y
%store -r Z
xth=X.reshape(-1)
yth=Y.reshape(-1)
zth=Z.reshape(-1)
d_matrix = np.sqrt(X**2+Y**2+Z**2)
dth=d_matrix.reshape(-1)

In [ ]:
frames=[]

camera = dict(
    up=dict(x=0, y=0, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=0, z=0)
)


fig9 = go.Figure()

# fig9.update_scenes(camera=camera)


fig9 = make_subplots(rows=1, cols=2,
                     subplot_titles=('Experiment', 'Theory'),
                     specs=[[{"type": "mesh3d"}, {"type": "surface"}]],)


fig9.add_trace(go.Mesh3d(x=xgo, y=ygo, z=zgo,i=tigo, j=tjgo, k=tkgo, intensity=dgo,
                        coloraxis="coloraxis1",
                        #    colorscale="Viridis",
                     #       colorbar_len=0.75,
                     #       flatshading=True,
                           lighting=dict(ambient=0.5,diffuse=1,fresnel=4,specular=0.5,roughness=0.05,facenormalsepsilon=0,vertexnormalsepsilon=0),lightposition=dict(x=100,y=100,z=1000)),1,1)
        
fig9.add_trace(go.Surface(x=X, y=Y, z=Z,
                          surfacecolor=d_matrix,
                          connectgaps=True,
                        coloraxis="coloraxis2",

                        #   colorscale="Viridis",
                     #       colorbar_len=0.75,
                           lighting=dict(ambient=0.5,diffuse=1,fresnel=4,specular=0.5,roughness=0.05),lightposition=dict(x=100,y=100,z=1000)),1,2)



T = np.arange(0,  2*np.pi, 0.125)
# T = np.arange(0,  2*np.pi, 0.0625)
xyzm = np.stack((xgo, ygo, zgo))
xyzs = np.stack((X.reshape(-1), Y.reshape(-1), Z.reshape(-1)))
for alpha in T:
    xrm, yrm, zrm  =  np.einsum('ik, kj -> ij', rot3d(alpha,0.25*np.pi,0.), xyzm)#  batch rotation of all points (x,y,z) on the Mesh
    xrs, yrs, zrs  =  np.einsum('ik, kj -> ij', rot3d(alpha,0.25*np.pi,0.), xyzs)#  batch rotation of all points (x,y,z) on the Mesh
    dths= np.sqrt(xrs**2+xrs**2+zrs**2)
    frames.append(go.Frame(data=[go.Mesh3d(x=xrm, y=yrm, z=zrm),
                                 go.Surface(x=xrs.reshape(200,100), 
                                            y=yrs.reshape(200,100),
                                            z=zrs.reshape(200,100),
                                            surfacecolor=d_matrix)],
                           traces=[0,1])) 
fig9.update(frames=frames)

fig9.update_layout(scene=dict(xaxis_range=[-46,46], xaxis_visible=False, xaxis_autorange=False,
                              yaxis_range=[-46,46], yaxis_visible=False, yaxis_autorange=False,
                              zaxis_range=[-46,46], zaxis_visible=False, zaxis_autorange=False),
                  scene2=dict(xaxis_range=[-0.007, 0.007], xaxis_visible=False, xaxis_autorange=False,
                              yaxis_range=[-0.007, 0.007], yaxis_visible=False, yaxis_autorange=False,
                              zaxis_range=[-0.007, 0.007], zaxis_visible=False, zaxis_autorange=False),
                  coloraxis1=dict(colorscale='Viridis', colorbar=dict(x=-.1, xanchor='left', thickness=20)),
                  coloraxis2=dict(colorscale='Viridis', colorbar=dict(x=1.05, xanchor='right', thickness=20)),

                  margin=dict(l=0, r=0, t=0, b=0),

                  # scene2=dict(xaxis2=dict(range=[-0.002, 0.002], visible=False, autorange=False),
                  #             yaxis2=dict(range=[-0.002, 0.002], visible=False, autorange=False),
                  #             zaxis2=dict(range=[-0.002, 0.002], visible=False, autorange=False)),
                                             
                  updatemenus=[dict(type='buttons',
                  showactive=False,
                  y=0.7,
                  x=-0.15,
                  xanchor='left',
                  yanchor='bottom',
                  buttons=[dict(label='Play', method='animate', args=[None, dict(frame=dict(duration=0, redraw=True), 
                                                                     transition=dict(duration=0,easing="quadratic-in-out"),
                                                                     fromcurrent=True,
                                                                  #    mode='immediate'
                                                                  )]),
                           dict(label='Pause', method='animate', args=[None, dict(frame=dict(duration=0, redraw=False), 
                                                                      mode='immediate')])


                          ]
                      )]
                  )

# fig9.show()
plotly.offline.plot(fig9, filename=r"../PYTHON_graphs/OUTPUTS/3D_"+CH+"_"+en+"_exp.html")

In [ ]:
print(fig9.layout)